In [1]:
%matplotlib qt
import os
from os.path import join as pjoin
import numpy as np
import hyperspy.api as hs
# import fpd.fpd_processing as fpdp
import pyxem as pxm
import matplotlib.pyplot as plt

from glob import glob
from skimage.filters import threshold_otsu
from skimage.morphology import binary_dilation, binary_erosion, area_closing

import scipy.constants as sc

In [2]:
# Create output directory for processed files; input directory from DPC raw processing
path_input = "20241029_STEM"
path_output = "003_dpc_cor_data"
os.makedirs(path_output, exist_ok=True)

f1 = '20241029_STEM/20241029_114148/512x512_LMSTEM_dosetest_Ga_30x30um_sq2.hspy'
f2 = '20241029_STEM/20241029_122214/512x512_LMSTEM_dosetest4_Ga_20x20um_sq3.hspy'


In [3]:
# Change this variable from either "com" (center of mass) or "pc" (phase correlation)
process_type = 'com'

# Load the file, either sq2 or sq3
fname = f2
s = hs.load(fname, lazy=True)

#### For 500 and 1000 

Look at the beam 

In [4]:
s_1000 = hs.load(f1, lazy=True)
s_500 = hs.load(f2, lazy=True)

In [ ]:
s_500.plot(norm='symlog')

In [146]:
s_1000.plot(norm='symlog')

[########################################] | 100% Completed | 47.64 s


In [5]:
s_1000_bs = s_1000.get_direct_beam_position(method="center_of_mass")
s_1000_vbf = s_1000.sum(axis=(-1, -2)).T
s_500_bs = s_500.get_direct_beam_position(method="center_of_mass")
s_500_vbf = s_500.sum(axis=(-1, -2)).T

s_1000_bs.compute()
s_1000_vbf.compute()
s_500_bs.compute()
s_500_vbf.compute()

[########################################] | 100% Completed | 102.74 s
[########################################] | 100% Completed | 52.75 s
[########################################] | 100% Completed | 84.08 s
[########################################] | 100% Completed | 50.60 s


In [6]:
# Assuming dimensions from <ElectronDiffraction2D> object
dimensions = (512, 513)

mask1000 = np.zeros(dimensions, dtype=bool)
mask500 = np.zeros(dimensions, dtype=bool)
mask1000[20:420, 0:410] = True # f1 [y, x]
mask500[47:409, 97:449] = True # f2 [y, x]

# linear plane
s_1000_bs_lp = s_1000_bs.get_linear_plane(mask=mask1000)
s_500_bs_lp = s_500_bs.get_linear_plane(mask=mask500)

# dscan corrected
s_1000_bs_cor = s_1000_bs - s_1000_bs_lp
s_500_bs_cor = s_500_bs - s_500_bs_lp

# Calibration factor in mrad/px
calibration_factor = 0.009328152465138774  # mrad/px

# Define scale in µm/px
scale = 2.1 / 42  # µm/px

# Desired scalebar length in µm and pixels
desired_length_um = 5  # µm
desired_length_px = (desired_length_um / scale)

# Extract dpcx (x-shift) and dpcy (y-shift) signals
dpcx_1000 = s_1000_bs_cor.data[:, :, 0] * calibration_factor * 1000  # Extract x-shift (first channel) * calibration factor * convert to urad
dpcy_1000 = s_1000_bs_cor.data[:, :, 1] * calibration_factor * 1000 # Extract y-shift (second channel) * calibration factor * convert to urad

dpcx_500 = s_500_bs_cor.data[:, :, 0] * calibration_factor * 1000  # Extract x-shift (first channel) * calibration factor * convert to urad
dpcy_500 = s_500_bs_cor.data[:, :, 1] * calibration_factor * 1000 # Extract y-shift (second channel) * calibration factor * convert to urad

Plot DPCx and DPCy

In [158]:
from matplotlib_scalebar.scalebar import ScaleBar

vmin, vmax = -30, 30

# Plot dpcx
plt.figure(figsize=(8, 6))
plt.imshow(dpcy_500, cmap='gray', vmax=vmax, vmin=vmin)
plt.colorbar(label='$\mu$rad')
plt.title('DPCy Signal')
plt.axis('off')


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white', 'size': 14}  # Text properties for the label
scalebar.linewidth = 20  # Adjust scalebar line width
scalebar.frameon = False  # Turn off the background box
#scalebar.label = f"{desired_length_um} µm"
plt.gca().add_artist(scalebar)


plt.show()

Plot VBF

In [159]:
# Plot VBF
plt.figure(figsize=(8, 6))
plt.imshow(s_1000_vbf.data, cmap='gray')
plt.title('VBF')
plt.axis('off')


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white'}  # Text properties for the label
scalebar.frameon = False  # Turn off the background box
#scalebar.label = f"{desired_length_um} µm"
plt.gca().add_artist(scalebar)


plt.show()

## As done before

In [7]:
s_bs = s.get_direct_beam_position(method="center_of_mass")
s_vbf = s.sum(axis=(-1, -2)).T

In [8]:
s_vbf.compute()
s_bs.compute()

s_vbf.change_dtype("float64")
s_bs.change_dtype("float64")

[########################################] | 100% Completed | 121.03 s
[########################################] | 100% Completed | 139.02 s


In [65]:
s_bs.plot()

[<Axes: title={'center': 'x-shift'}, xlabel=' axis ()', ylabel=' axis ()'>,
 <Axes: title={'center': 'y-shift'}, xlabel=' axis ()', ylabel=' axis ()'>]

In [ ]:
s_vbf.save("001_data/" + fname.replace(".hspy", "_vbf.hspy"), overwrite=True)
s_bs.save("001_data/" + fname.replace(".hspy", "_bs.hspy"), overwrite=True)

# Perform d-scan correction

In [ ]:
#  add a mask before d-scan correction; we wish to mask out magnetic regions

In [9]:
# Assuming dimensions from <ElectronDiffraction2D> object
dimensions = (512, 513)

# Initialize the mask with all False
# Mask for sq2
mask = np.zeros(dimensions, dtype=bool)

# Define the region to set as True
# For example, let's mask a rectangular area 
# mask[20:420, 0:410] = True # f1 [y, x]
mask[47:409, 97:449] = True # f2 [y, x]

In [121]:
# Plot the mask
plot = True
if plot:
    plt.figure(figsize=(8, 8))
    plt.imshow(mask.astype(float), cmap='gray')  # Convert mask to float for visualization
    plt.title("Mask Visualization")
    plt.xlabel("X-axis (columns)")
    plt.ylabel("Y-axis (rows)")
    plt.colorbar(label='Mask Value')  # Add a color bar for clarity
    plt.show()

In [10]:
# linear plane
s_bs_lp = s_bs.get_linear_plane(mask=mask)
s_bs_lp.plot()

[<Axes: title={'center': 'x-shift'}, xlabel=' axis ()', ylabel=' axis ()'>,
 <Axes: title={'center': 'y-shift'}, xlabel=' axis ()', ylabel=' axis ()'>]

In [11]:
# dscan corrected
s_bs_cor = s_bs - s_bs_lp
s_bs_cor.plot()

[<Axes: title={'center': 'x-shift'}, xlabel=' axis ()', ylabel=' axis ()'>,
 <Axes: title={'center': 'y-shift'}, xlabel=' axis ()', ylabel=' axis ()'>]

#### Cropped 

In [90]:
s_crop = s.inav[60:130, 350:420]
s_crop_bs = s_crop.get_direct_beam_position(method="center_of_mass")
s_crop_bs.compute()
s_crop_bs.change_dtype("float64")
s_crop_bs_lp = s_crop_bs.get_linear_plane()
s_crop_bs_cor = s_crop_bs - s_crop_bs_lp
#s_crop_bs_cor.plot()

s_crop_dpc = s_crop_bs_cor.get_color_signal()
s_crop_dpc.plot(scalebar=False, axes_off=True)

s_crop.calibrate()
# Here I want to turn of colorbar, titles, axis, and scalebar

# Here I want to code such that we take a probe image of three different chosen positions

# Then a subplot with one big plot of s_crop_dpc, then three probe positions on the side of that. The three probe images should be quadratic and have in total the height of the dpc image.


[########################################] | 100% Completed | 1.50 sms
[                                        ] | 0% Completed | 126.07 us

/Users/thyra/miniforge3/envs/hyperspy2024/lib/python3.11/site-packages/pyxem/signals/beam_shift.py:466: VisibleDeprecationWarning: Function `get_color_signal()` is deprecated and will be removed in version 1.0.0. Use `get_magnitude_phase_signal()` instead.
  since="0.19.0",


[########################################] | 100% Completed | 943.65 ms


In [ ]:
# Crop uninteresting detector regions to ease processing
detector_slice = np.s_[97:157,:]
#s_crop_crop = s_crop.isig[detector_slice]
#s_crop_crop.plot()

In [109]:
s_crop.isig[detector_slice].plot(navigator=s_crop_bs_cor.isig[0].T)

In [116]:
s_crop_dpc.data

array([[( 3869,  1958,     0), ( 3113, 15418,     0),
        ( 3856,  5879,     0), ..., ( 1625,  4699,     0),
        ( 3254,     0,  5383), (13458,     0, 13428)],
       [(    0,  2931,  2299), (    0,    32,  3057),
        (   77,     0, 10579), ..., ( 1338,  7672,     0),
        ( 8264,     0,  7112), (11464, 12363,     0)],
       [( 7714,  5615,     0), ( 1360,  4025,     0),
        (30130,     0,  9255), ..., ( 5387,   174,     0),
        (19136,     0, 16785), (    0,  3835, 13905)],
       ...,
       [( 3427,  2415,     0), (14700,     0, 19376),
        (18875,     0, 40399), ..., ( 1646,  6895,     0),
        (18369,     0,  5012), ( 7595,  9426,     0)],
       [( 1939, 17496,     0), ( 4988,  2243,     0),
        (11948,  3441,     0), ..., ( 3612,     0, 11359),
        (16448,     0, 27643), (    0, 29574, 13238)],
       [( 6406,     0,  2477), ( 1097,  8151,     0),
        (12170,  8878,     0), ..., (12966,     0, 20332),
        (    0, 12888,  7147), ( 20

In [119]:
# Convert to a 3D array (shape: [height, width, 3])
rgb_array = np.stack([s_crop_dpc.data['R'], s_crop_dpc.data['G'], s_crop_dpc.data['B']], axis=-1).astype(np.float64)

# Apply Gaussian filter to each channel
smoothed_rgb = np.zeros_like(rgb_array)
for i in range(3):  # Loop through RGB channels
    smoothed_rgb[..., i] = gaussian_filter(rgb_array[..., i], sigma=1.0)

# Convert back to structured array if necessary
smoothed_structured = np.zeros_like(s_crop_dpc.data)
smoothed_structured['R'] = smoothed_rgb[..., 0].astype(s_crop_dpc.data.dtype['R'])
smoothed_structured['G'] = smoothed_rgb[..., 1].astype(s_crop_dpc.data.dtype['G'])
smoothed_structured['B'] = smoothed_rgb[..., 2].astype(s_crop_dpc.data.dtype['B'])

# Convert structured array to a 3D RGB array (scale to [0, 1] for visualization)
rgb_image = np.stack([
    smoothed_structured['R'],
    smoothed_structured['G'],
    smoothed_structured['B']
], axis=-1).astype(np.float64)

# Normalize the values to range [0, 1] (required for proper visualization with imshow)
rgb_image /= np.max(rgb_image)


plt.figure(figsize=(8, 8))
plt.imshow(rgb_image)
plt.axis('off')  # Optional: Turn off axis labels
plt.title("DPC with Gaussian Filter")
plt.show()




###### dette er virrvarr

In [88]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Turn off colorbar, titles, axes, and scalebar
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
s_crop.plot(navigator=s_crop_bs_cor.isig[0].T, ax=ax1)
ax1.axis('off')  # Turn off axes
ax1.set_title("")  # Remove title
plt.show()

# Define probe positions (example positions, adjust as needed)
probe_positions = [(10, 10), (20, 20), (30, 30)]  # (row, column)

# Extract probe images at the chosen positions
probe_images = [s_crop.inav[row, col].data for row, col in probe_positions]

# Create a subplot layout
fig = plt.figure(figsize=(8, 8))
gs = GridSpec(1, 4, width_ratios=[3, 1, 1, 1], wspace=0.1)  # Big DPC + 3 vertical small plots

# Big plot for `s_crop_dpc`
ax_dpc = fig.add_subplot(gs[0])
s_crop_dpc.plot(ax=ax_dpc)
ax_dpc.axis('off')  # Turn off axes
ax_dpc.set_title("")  # Remove title

# Smaller plots for probe images
for i, probe_img in enumerate(probe_images):
    ax_probe = fig.add_subplot(gs[i + 1])
    ax_probe.imshow(probe_img, cmap='gray', interpolation='nearest')  # Show as grayscale
    ax_probe.axis('off')  # Turn off axes

plt.show()


In [ ]:
# Define probe positions (row, column)
probe_positions = [(10, 10), (20, 20), (30, 30)]

# Extract probe images at the chosen positions
probe_images = [s_crop.inav[row, col].data.squeeze() for row, col in probe_positions]

# Create a figure for the vertical subplot
fig, axes = plt.subplots(len(probe_images), 1, figsize=(4, 12))  # Adjust figure size as needed

# Plot each probe image in its respective subplot
for i, (ax, probe_img) in enumerate(zip(axes, probe_images)):
    ax.imshow(probe_img, cmap='gray', interpolation='nearest')
    ax.axis('off')  # Turn off the axes
    ax.set_title(f"Probe {i + 1}", fontsize=12)

# Tighten layout for better spacing
plt.tight_layout()
plt.show()


### Finding mrad/px

In [ ]:
s1 = s.inav[0,0]
s1.calibrate() 

[########################################] | 100% Completed | 108.19 ms


In [ ]:
# mrad/px ratio is then
calibration_factor = s1.axes_manager[-1].scale 

# calibration_factor = 0.009328152465138774 # is a scale used for 20241029_114148 data

### Finding nm/px

In [ ]:
# s.calibrate() 

### DPCx and DPCy images

In [16]:
s_bs_cor

<BeamShift, title: , dimensions: (513, 512|2)>

In [17]:
# Calibration factor in mrad/px
calibration_factor = 0.009328152465138774  # mrad/px

# Define scale in µm/px
scale = 2.1 / 42  # µm/px

# Desired scalebar length in µm and pixels
desired_length_um = 5  # µm
desired_length_px = (desired_length_um / scale)

# Extract dpcx (x-shift) and dpcy (y-shift) signals
dpcx = s_bs_cor.data[:, :, 0] * calibration_factor * 1000  # Extract x-shift (first channel) * calibration factor * convert to urad
dpcy = s_bs_cor.data[:, :, 1] * calibration_factor * 1000 # Extract y-shift (second channel) * calibration factor * convert to urad

In [18]:
from matplotlib_scalebar.scalebar import ScaleBar

# Plot dpcx
plt.figure(figsize=(8, 6))
plt.imshow(dpcx, cmap='gray')
plt.colorbar(label='$\mu$rad')
plt.title('DPCx Signal')
plt.axis('off')


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white'}  # Text properties for the label
scalebar.frameon = False  # Turn off the background box
#scalebar.label = f"{desired_length_um} µm"
plt.gca().add_artist(scalebar)


plt.show()

In [19]:
# Plot dpcy
plt.figure(figsize=(8, 6))
plt.imshow(dpcy, cmap='gray')
plt.colorbar(label='$\mu$rad')
plt.title('DPCy')
plt.axis('off')


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white'}  # Text properties for the label
scalebar.frameon = False  # Turn off the background box
#scalebar.label = f"{desired_length_um} µm"
plt.gca().add_artist(scalebar)


plt.show()

### Virtual Bright Field Image

In [20]:
# Plot VBF
plt.figure(figsize=(8, 6))
plt.imshow(s_vbf.data, cmap='gray')
plt.title('VBF')
plt.axis('off')


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white'}  # Text properties for the label
scalebar.frameon = False  # Turn off the background box
#scalebar.label = f"{desired_length_um} µm"
plt.gca().add_artist(scalebar)


plt.show()

### Magnitude signal

#### functions

In [12]:
def beta_to_bst(beam_deflection, acceleration_voltage):
    """Calculate Bs * t values from beam deflection (beta).

    Parameters
    ----------
    beam_deflection : NumPy array
        In radians
    acceleration_voltage : float
        In Volts

    Returns
    -------
    Bst : NumPy array
        In Tesla * meter

    Examples
    --------
    >>> import numpy as np
    >>> import pixstem.dpc_tools as dpct
    >>> data = np.random.random((100, 100))  # In radians
    >>> acceleration_voltage = 200000  # 200 kV (in Volt)
    >>> bst = dpct.beta_to_bst(data, 200000)

    """
    wavelength = acceleration_voltage_to_wavelength(acceleration_voltage)
    beta = beam_deflection
    e = sc.elementary_charge
    h = sc.Planck
    Bst = beta*h/(wavelength*e)
    return Bst

def acceleration_voltage_to_wavelength(acceleration_voltage):
    """Get electron wavelength from the acceleration voltage.

    Parameters
    ----------
    acceleration_voltage : float or array-like
        In Volt

    Returns
    -------
    wavelength : float or array-like
        In meters

    Examples
    --------
    >>> import pixstem.diffraction_tools as dt
    >>> acceleration_voltage = 200000  # 200 kV (in Volt)
    >>> wavelength = dt.acceleration_voltage_to_wavelength(
    ...     acceleration_voltage)
    >>> wavelength_picometer = wavelength*10**12

    """
    E = acceleration_voltage*sc.elementary_charge
    h = sc.Planck
    m0 = sc.electron_mass
    c = sc.speed_of_light
    wavelength = h/(2*m0*E*(1 + (E/(2*m0*c**2))))**0.5
    return wavelength

def tesla_to_am(data):
    """Convert data from Tesla to A/m

    Parameters
    ----------
    data : NumPy array
        Data in Tesla

    Returns
    -------
    output_data : NumPy array
        In A/m

    Examples
    --------
    >>> import numpy as np
    >>> import pixstem.dpc_tools as dpct
    >>> data_T = np.random.random((100, 100))  # In tesla
    >>> data_am = dpct.tesla_to_am(data_T)

    """
    return data/sc.mu_0



#### plot signal

In [13]:
s_magnitude_signal = s_bs_cor.get_magnitude_signal()
s_magnitude_unit = tesla_to_am(beta_to_bst(s_magnitude_signal, 200000)) # with correct unit, but it is something wierd.

[########################################] | 100% Completed | 774.81 ms


In [14]:
# Calculate the magnitude signal from the beam shift correction data
s_1000_magnitude_signal = s_1000_bs_cor.get_magnitude_signal()
s_1000_magnitude_unit = tesla_to_am(beta_to_bst(s_1000_magnitude_signal, 200000)) # with correct unit, but it is something wierd.

[########################################] | 100% Completed | 903.69 ms


Rotating image

In [126]:
from scipy.ndimage import rotate

In [161]:
s_magnitude_signal_rotate = s_magnitude_signal.map(rotate, angle = -5, reshape = False, inplace=False)

In [162]:
s_magnitude_signal_rotate.plot()

Plot magnitude

In [164]:
# Visualize the magnitude signal
plt.imshow(s_magnitude_signal.data, cmap='viridis')
plt.colorbar(label='Beam Shift Magnitude')
plt.show()

In [172]:
# Plot magnitude signal
plt.figure(figsize=(8, 6))
plt.imshow(s_1000_magnitude_unit.data, cmap='gray')
plt.colorbar(label='Beam Shift Magnitude') 
#plt.title('magnitude signal data')
plt.axis('off')


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white'}  # Text properties for the label
scalebar.frameon = False  # Turn off the background box
plt.gca().add_artist(scalebar)

plt.show()

#### Magnetic strength vs ion dose plot 1000 ns

In [211]:

# Define the y-coordinates for the ranges you want to average
y_start = [52, 119, 189, 253, 325, 394]
y_end = [62, 129, 199, 263, 335, 404]

# Define the x-coordinates for the ranges you want to average
x_start = [29, 95, 162, 230, 300, 365]
x_end = [39, 105, 172, 240, 310, 375]


# Initialize a list to store all the flipped line scans and their standard deviations
all_flipped_line_scans = []
all_flipped_line_scans_std = []

# Loop through each y-range and compute the averaged line scans
for y_s, y_e in zip(y_start, y_end):
    # Extract the data for the specified y-range and compute the average along the y-axis
    data_avg_y = np.mean(s_1000_magnitude_signal.data[y_s:y_e, 70:460], axis=0)

    # Initialize lists to store the averaged line scans and their standard deviations for each x-range
    line_scans_avg = []
    line_scans_std = []

    # Loop through each x-range and compute the average line scan and its standard deviation
    for start, end in zip(x_start, x_end):
        line_scan_avg = np.mean(data_avg_y[start:end])
        line_scan_std = np.std(data_avg_y[start:end])
        line_scans_avg.append(line_scan_avg)
        line_scans_std.append(line_scan_std)

    # Flip the line scans and their standard deviations
    flipped_line_scans = list(reversed(line_scans_avg))
    flipped_line_scans_std = list(reversed(line_scans_std))
    all_flipped_line_scans.extend(flipped_line_scans)
    all_flipped_line_scans_std.extend(flipped_line_scans_std)

# Flip the dwelltime array
dwelltime = 1000 * np.array([1.0, 0.9722222222222222, 0.9444444444444444, 0.9166666666666666,
                            0.8888888888888888, 0.8611111111111112, 0.8333333333333334,
                            0.8055555555555556, 0.7777777777777778, 0.75,
                            0.7222222222222222, 0.6944444444444444,
                            0.6666666666666666, 0.6388888888888888,
                            0.6111111111111112, 0.5833333333333334,
                            0.5555555555555556, 0.5277777777777778,
                            0.5, 0.4722222222222222,
                            0.4444444444444444, 0.4166666666666667,
                            0.3888888888888889, 0.3611111111111111,
                            0.3333333333333333, 0.3055555555555556,
                            0.2777777777777778, 0.25,
                            0.2222222222222222, 0.19444444444444445,
                            0.16666666666666666, 0.1388888888888889,
                            0.1111111111111111, 0.08333333333333333,
                            0.05555555555555555, 0.027777777777777776])
flipped_dwelltime = dwelltime[::-1]


In [201]:
# Another plot with errorbands 
plt.rcParams.update({
    'font.size': 12,               # General font size
    'axes.labelsize': 14,          # Font size for axis labels
    'axes.titlesize': 16,          # Font size for the title
    'xtick.labelsize': 12,         # Font size for x-axis tick labels
    'ytick.labelsize': 12,         # Font size for y-axis tick labels
    'lines.linewidth': 2,          # Line width
    'figure.figsize': (10, 6),     # Figure size
})

# Calculate the error band
flipped_dwelltime_truncated = flipped_dwelltime[:len(all_flipped_line_scans)]
lower_bound = np.array(all_flipped_line_scans) - np.array(all_flipped_line_scans_std)
upper_bound = np.array(all_flipped_line_scans) + np.array(all_flipped_line_scans_std)

# Create the plot
plt.figure()
plt.plot(flipped_dwelltime_truncated, all_flipped_line_scans, '-o', color='navy', label='Mean Intensity')
plt.fill_between(flipped_dwelltime_truncated, lower_bound, upper_bound, color='navy', alpha=0.2, label='Error Band')

# Customize the plot for publication
#plt.title('Magnetic Response to Increasing Ion Dose', pad=15)
plt.xlabel('Dwell Time [ns]', labelpad=10)
plt.ylabel('Relative magnetic strength [a.u.]', labelpad=10)
plt.legend(fontsize=12, loc='best')
plt.grid(True, linestyle='--', linewidth=0.5, color='gray')
plt.tight_layout()

# Save the plot as a high-resolution image
#plt.savefig('flipped_line_scans_with_errorband.png', dpi=600, bbox_inches='tight')
plt.show()

In [199]:
import matplotlib.pyplot as plt
import numpy as np


# Define the y-coordinates for the ranges you want to highlight
y_start = [52, 119, 189, 253, 325, 394]
y_end = [62, 129, 199, 263, 335, 404]

# Define the x-coordinates for the ranges you want to highlight
x_start = [29, 95, 162, 230, 300, 365]
x_end = [39, 105, 172, 240, 310, 375]

# Plot the 2D signal
plt.figure(figsize=(10, 8))
plt.imshow(s_1000_magnitude_unit, cmap='gray', aspect='auto')
plt.colorbar(label='Signal Intensity')
plt.title('2D Signal with Highlighted Ranges')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')

# Overlay vertical lines for x_start and x_end
for y_s, y_e in zip(y_start, y_end):
    for x_s, x_e in zip(x_start, x_end):
        # Add vertical lines for x_start and x_end
        plt.vlines([x_s, x_e], y_s, y_e, colors='red', linestyles='--', linewidth=2)

# Customize the plot appearance
plt.grid(False)
plt.tight_layout()
plt.show()


In [181]:
s_1000_magnitude_unit

array([[0.7284055 , 0.95026738, 0.64768339, ..., 0.42974601, 0.16202927,
        0.23820636],
       [0.23709067, 0.87377926, 0.91704666, ..., 0.67488567, 0.93917176,
        0.97147591],
       [0.94398013, 0.25907859, 0.7833286 , ..., 0.38309275, 0.32902812,
        0.40954188],
       ...,
       [0.49264292, 0.77430751, 0.86110004, ..., 0.1064125 , 0.75020207,
        0.37903505],
       [0.98565365, 0.96794109, 0.43893947, ..., 0.98964862, 0.57449016,
        0.48192729],
       [0.97734288, 0.67484993, 0.78141291, ..., 0.23487451, 0.25730217,
        0.44138664]])

In [195]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label, find_objects

# Thresholding to detect lighter squares
threshold = np.percentile(s_1000_magnitude_unit, 90)  # Adjust percentile to capture "lighter" regions
binary_mask = s_1000_magnitude_unit > threshold

# Label connected components in the binary mask
labeled_array, num_features = label(binary_mask)

# Get the bounding boxes of all labeled features
objects = find_objects(labeled_array)

# Filter objects by size and aspect ratio to isolate square-like regions
detected_squares = []
for obj in objects:
    y_slice, x_slice = obj
    height = y_slice.stop - y_slice.start
    width = x_slice.stop - x_slice.start

    # Check for approximate square-like regions
    aspect_ratio = width / float(height)
    if 0.9 <= aspect_ratio <= 1.1:  # Aspect ratio close to 1
        detected_squares.append((x_slice.start, y_slice.start, x_slice.stop, y_slice.stop))

# Sort detected squares to match a 6x6 grid order (row by row)
detected_squares = sorted(detected_squares, key=lambda c: (c[1], c[0]))

# Ensure we have exactly 36 squares for the 6x6 grid
if len(detected_squares) != 36:
    print(f"Warning: Detected {len(detected_squares)} regions, not 36. Adjust threshold or filtering criteria!")

# Visualize the detected squares
plt.figure(figsize=(10, 8))
plt.imshow(s_1000_magnitude_unit, cmap="viridis", aspect="auto")
plt.colorbar(label="Intensity")
plt.title("Detected Squares Fitted to 6x6 Matrix")

# Overlay rectangles around detected squares and label them
for idx, (x_start, y_start, x_end, y_end) in enumerate(detected_squares):
    plt.gca().add_patch(plt.Rectangle((x_start, y_start), x_end - x_start, y_end - y_start, 
                                       edgecolor="red", facecolor="none", linewidth=2))
    plt.text(x_start + (x_end - x_start) // 2, y_start + (y_end - y_start) // 2, f"{idx+1}",
             color="yellow", fontsize=8, ha="center", va="center")

plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.tight_layout()
plt.show()

# Output the coordinates of the 6x6 grid
print("Coordinates of Detected Squares (x_start, y_start, x_end, y_end):")
for idx, coords in enumerate(detected_squares):
    print(f"Square {idx+1}: {coords}")


Coordinates of Detected Squares (x_start, y_start, x_end, y_end):
Square 1: (86, 0, 87, 1)
Square 2: (427, 0, 428, 1)
Square 3: (168, 1, 169, 2)
Square 4: (264, 1, 265, 2)
Square 5: (289, 1, 290, 2)
Square 6: (308, 1, 309, 2)
Square 7: (350, 1, 352, 3)
Square 8: (178, 2, 179, 3)
Square 9: (373, 2, 374, 3)
Square 10: (262, 3, 263, 4)
Square 11: (309, 3, 310, 4)
Square 12: (381, 3, 382, 4)
Square 13: (398, 3, 399, 4)
Square 14: (459, 3, 460, 4)
Square 15: (166, 4, 167, 5)
Square 16: (220, 4, 221, 5)
Square 17: (399, 4, 400, 5)
Square 18: (435, 4, 436, 5)
Square 19: (453, 4, 454, 5)
Square 20: (28, 5, 29, 6)
Square 21: (127, 5, 128, 6)
Square 22: (192, 5, 193, 6)
Square 23: (309, 5, 311, 7)
Square 24: (318, 5, 319, 6)
Square 25: (437, 5, 438, 6)
Square 26: (445, 5, 446, 6)
Square 27: (471, 5, 472, 6)
Square 28: (83, 6, 84, 7)
Square 29: (150, 6, 151, 7)
Square 30: (295, 6, 296, 7)
Square 31: (116, 7, 117, 8)
Square 32: (182, 7, 184, 9)
Square 33: (197, 7, 198, 8)
Square 34: (488, 7, 489, 

#### Magnetic strength vs ion dose plot 500 ns

In [15]:
# Define the y-coordinates for the ranges you want to average

y_start = [50, 105, 170, 235, 300, 365]
y_end = [80, 135, 200, 265, 330, 395]

# Define the x-coordinates for the ranges you want to average
x_start = [18, 84, 145, 209, 271, 336]
x_end = [53, 116, 180, 245, 307, 370]

"""
# Define the y-coordinates for the ranges you want to average
y_start = [49, 115, 186, 251, 321, 391]
y_end = [72, 142, 210, 279, 348, 416]

# Define the x-coordinates for the ranges you want to average
x_start = [25, 95, 160, 229, 295, 363]
x_end = [44, 111, 177, 248, 315, 379]
"""

# Initialize a list to store all the flipped line scans and their standard deviations
all_flipped_line_scans = []
all_flipped_line_scans_std = []

# Loop through each y-range and compute the averaged line scans
for y_s, y_e in zip(y_start, y_end):
    # Extract the data for the specified y-range and compute the average along the y-axis
    data_avg_y = np.mean(s_magnitude_signal.data[y_s:y_e, 70:460], axis=0)

    # Initialize lists to store the averaged line scans and their standard deviations for each x-range
    line_scans_avg = []
    line_scans_std = []

    # Loop through each x-range and compute the average line scan and its standard deviation
    for start, end in zip(x_start, x_end):
        line_scan_avg = np.mean(data_avg_y[start:end])
        line_scan_std = np.std(data_avg_y[start:end])
        line_scans_avg.append(line_scan_avg)
        line_scans_std.append(line_scan_std)

    # Flip the line scans and their standard deviations
    flipped_line_scans = list(reversed(line_scans_avg))
    flipped_line_scans_std = list(reversed(line_scans_std))
    all_flipped_line_scans.extend(flipped_line_scans)
    all_flipped_line_scans_std.extend(flipped_line_scans_std)

# Flip the dwelltime array
dwelltime = 500 * np.array([1.0, 0.9722222222222222, 0.9444444444444444, 0.9166666666666666,
                            0.8888888888888888, 0.8611111111111112, 0.8333333333333334,
                            0.8055555555555556, 0.7777777777777778, 0.75,
                            0.7222222222222222, 0.6944444444444444,
                            0.6666666666666666, 0.6388888888888888,
                            0.6111111111111112, 0.5833333333333334,
                            0.5555555555555556, 0.5277777777777778,
                            0.5, 0.4722222222222222,
                            0.4444444444444444, 0.4166666666666667,
                            0.3888888888888889, 0.3611111111111111,
                            0.3333333333333333, 0.3055555555555556,
                            0.2777777777777778, 0.25,
                            0.2222222222222222, 0.19444444444444445,
                            0.16666666666666666, 0.1388888888888889,
                            0.1111111111111111, 0.08333333333333333,
                            0.05555555555555555, 0.027777777777777776])
flipped_dwelltime = dwelltime[::-1]


In [16]:
# Another plot with errorbands 
plt.rcParams.update({
    'font.size': 12,               # General font size
    'axes.labelsize': 14,          # Font size for axis labels
    'axes.titlesize': 16,          # Font size for the title
    'xtick.labelsize': 12,         # Font size for x-axis tick labels
    'ytick.labelsize': 12,         # Font size for y-axis tick labels
    'lines.linewidth': 2,          # Line width
    'figure.figsize': (8, 10),     # Figure size
})

# Calculate the error band
flipped_dwelltime_truncated = flipped_dwelltime[:len(all_flipped_line_scans)]
lower_bound = np.array(all_flipped_line_scans) - np.array(all_flipped_line_scans_std)
upper_bound = np.array(all_flipped_line_scans) + np.array(all_flipped_line_scans_std)

# Create the plot
plt.figure()
plt.plot(flipped_dwelltime_truncated, all_flipped_line_scans, '-o', color='navy', label='Mean Intensity')
plt.fill_between(flipped_dwelltime_truncated, lower_bound, upper_bound, color='navy', alpha=0.2, label='Error Band')

# Customize the plot for publication
#plt.title('Magnetic Response to Increasing Ion Dose', pad=15)
plt.xlabel('Dwell Time [ns]', labelpad=10)
plt.ylabel('Relative magnetic strength [a.u.]', labelpad=10)
plt.legend(fontsize=12, loc='best')
plt.grid(True, linestyle='--', linewidth=0.5, color='gray')
plt.tight_layout()

# Save the plot as a high-resolution image
#plt.savefig('flipped_line_scans_with_errorband.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define the logistic function
def logistic(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))

# Perform the curve fitting
popt, pcov = curve_fit(logistic, flipped_dwelltime_truncated, all_flipped_line_scans, 
                       p0=[max(all_flipped_line_scans), 1, np.median(flipped_dwelltime_truncated)])

# Extract fitted parameters
L, k, x0 = popt

# Generate fitted curve
x_fit = np.linspace(min(flipped_dwelltime_truncated), max(flipped_dwelltime_truncated), 500)
y_fit = logistic(x_fit, *popt)

# Plot with curve fit
plt.rcParams.update({
    'font.size': 18,               # General font size
    'axes.labelsize': 20,          # Font size for axis labels
    'axes.titlesize': 16,          # Font size for the title
    'xtick.labelsize': 16,         # Font size for x-axis tick labels
    'ytick.labelsize': 16,         # Font size for y-axis tick labels
    'lines.linewidth': 2,          # Line width
    'figure.figsize': (8, 10),     # Figure size
})

# Calculate the error band
lower_bound = np.array(all_flipped_line_scans) - np.array(all_flipped_line_scans_std)
upper_bound = np.array(all_flipped_line_scans) + np.array(all_flipped_line_scans_std)

# Create the plot
plt.figure()
plt.plot(flipped_dwelltime_truncated, all_flipped_line_scans, '-o', color='navy', label='Mean Intensity')
plt.fill_between(flipped_dwelltime_truncated, lower_bound, upper_bound, color='navy', alpha=0.2, label='Error Band')
plt.plot(x_fit, y_fit, '--', color='red', label='Curve Fit')  # Fitted curve with dashed style, looks like a cumulative distribution function

# Customize the grid
plt.grid(visible=True, which='major', linestyle='--', linewidth=1, color='gray', alpha=0.3)
plt.grid(visible=True, which='minor', linestyle=':', linewidth=0.5, color='gray', alpha=0.5)
plt.minorticks_on()  # Enable minor ticks for a finer grid

# Customize the plot for publication
plt.xlabel('Dwell Time [ns]', labelpad=10)
plt.ylabel('Relative magnetic strength [a.u.]', labelpad=10)
plt.legend(fontsize=18, loc='best')
plt.tight_layout()

# Show the plot
plt.show()

/var/folders/2p/vhcm8hcs3d1fmd4xtjljf4fw0000gn/T/ipykernel_23127/1672593749.py:7: RuntimeWarning: overflow encountered in exp
  return L / (1 + np.exp(-k * (x - x0)))


In [18]:
plt.savefig("DT_graph_with_curvefit_biggerfont.png", dpi=300) 

### Magnitude phase signal

In [130]:
s_magnitude_phase = s_bs_cor.get_magnitude_phase_signal()
s_magnitude_phase.plot(scalebar=False, axes_off=True)

In [138]:
# s_magnitude_phase_rotate = s_magnitude_phase.map(rotate, angle = -5, reshape = False, inplace=False) # does not work

In [210]:
from pyxem.utils.plotting import plot_beam_shift_color

# Create a single figure
fig = plt.figure(figsize=(8, 8))  # Increased figure size for better proportions

# Add the main axis for the beam shift plot
ax_main = fig.add_axes([0.1, 0.1, 0.7, 0.7])  # Adjusted width to make space for a larger color indicator

# Add a separate axis for the color indicator
ax_color = fig.add_axes([0.65, 0.65, 0.15, 0.15])  

# Plot the beam shift with the color indicator
plot_beam_shift_color(ax=ax_main, signal=s_1000_bs_cor, autolim=False, magnitude_limits=(-1.5, 1.5), ax_indicator=ax_color)


# Add a scalebar
scalebar = ScaleBar(scale, units="µm", dimension="si-length")
scalebar.location = 'lower right'
scalebar.length = desired_length_um  # Set the scalebar length in µm
scalebar.color = 'white'  # Color of the scalebar line
scalebar.text_props = {'color': 'white'}  # Text properties for the label
scalebar.frameon = False  # Turn off the background box
ax_main.add_artist(scalebar)
# Show the figure
plt.show()



## Gaussian Filter

In [122]:
s_color = s_bs_cor.get_color_signal()

# Convert to a 3D array (shape: [height, width, 3])
rgb_array = np.stack([s_color.data['R'], s_color.data['G'], s_color.data['B']], axis=-1).astype(np.float64)

# Apply Gaussian filter to each channel
smoothed_rgb = np.zeros_like(rgb_array)
for i in range(3):  # Loop through RGB channels
    smoothed_rgb[..., i] = gaussian_filter(rgb_array[..., i], sigma=1.0)

# Convert back to structured array if necessary
smoothed_structured = np.zeros_like(s_color.data)
smoothed_structured['R'] = smoothed_rgb[..., 0].astype(s_color.data.dtype['R'])
smoothed_structured['G'] = smoothed_rgb[..., 1].astype(s_color.data.dtype['G'])
smoothed_structured['B'] = smoothed_rgb[..., 2].astype(s_color.data.dtype['B'])

# Convert structured array to a 3D RGB array (scale to [0, 1] for visualization)
rgb_image = np.stack([
    smoothed_structured['R'],
    smoothed_structured['G'],
    smoothed_structured['B']
], axis=-1).astype(np.float64)

# Normalize the values to range [0, 1] (required for proper visualization with imshow)
rgb_image /= np.max(rgb_image)


plt.figure(figsize=(8, 8))
plt.imshow(rgb_image)
plt.axis('off')  # Optional: Turn off axis labels
plt.title("DPC with Gaussian Filter")
plt.show()

/Users/thyra/miniforge3/envs/hyperspy2024/lib/python3.11/site-packages/pyxem/signals/beam_shift.py:466: VisibleDeprecationWarning: Function `get_color_signal()` is deprecated and will be removed in version 1.0.0. Use `get_magnitude_phase_signal()` instead.
  since="0.19.0",


## Div ting

### Rotating?

In [ ]:
s_dummy = pxm.data.dummy_data.get_simple_beam_shift_signal()
s_rot = s_dummy.rotate_data(10)
s_rot.plot()

### Ulike plots

s_bs_cor.get_phase_signal().plot()

In [ ]:
s_bs_cor.get_phase_signal().plot()